# Seq2Seq Example. - building Kor-En Translator

## Strategy
- Pretrain Decoder - Endcoder for each dataset (Korean, English)
  - Korean Encoder Training
  - English Decoder Training
  - English Encoder Training
  - Korean Decoder Training
- End-to-End Training - parallel corpus text dataset
  - Korean - English Translator
  - English - Korean Translator
- 


In [1]:
from model import GRUEncoder
from data import KoEnParallel, Korean, English
from transformers import AutoTokenizer, GPT2TokenizerFast
from torch.utils import data

# ko_tokenizer = AutoTokenizer.from_pretrained("skt/ko-gpt-trinity-1.2B-v0.5")
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

ko_dataset = English(tokenizer, 1024, 'train')
dataloader = data.DataLoader(ko_dataset, batch_size=10, collate_fn=ko_dataset)
ko_encoder = GRUEncoder(len(tokenizer.vocab), 1024, 1024, 2, padding_id=tokenizer.pad_token_id)

next(iter(dataloader))

Using custom data configuration Moo--korean-parallel-corpora-152c18962d41c571
Found cached dataset csv (/home/fritzprix/.cache/huggingface/datasets/Moo___csv/Moo--korean-parallel-corpora-152c18962d41c571/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/3 [00:00<?, ?it/s]

/home/fritzprix/miniconda3/envs/directml/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


tensor([[ 2.0045e+04,  2.8600e+02,  2.6140e+03,  1.4492e+04,  3.1800e+02,
          5.4600e+02,  3.6600e+02,  5.1710e+03,  3.4500e+02,  1.3530e+03,
          4.2800e+02,  1.7010e+03, -1.0000e+00, -1.0000e+00, -1.0000e+00,
         -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00,
         -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00,
         -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00,
         -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00,
         -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00,
         -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00,
         -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00,
         -1.0000e+00],
        [ 5.6800e+02,  2.5700e+02,  3.0680e+03,  2.5700e+02,  1.1780e+03,
          2.7450e+03,  2.0840e+03,  5.4600e+02,  2.5700e+02,  2.7978e+04,
          5.4000e+02,  1.2521e+04,  1.8480e+04,  1.0211e+04,  3.1810e+03,
          2.870

In [4]:
import pytorch_lightning as pl
from pytorch_lightning import loggers, callbacks

logger = loggers.TensorBoardLogger('logs')
model_checkout = callbacks.ModelCheckpoint('model/encoder', filename='model-{epoch}-{val_loss:.3f}')

trainer = pl.Trainer(logger=logger, accelerator='gpu', callbacks=[model_checkout])
trainer.fit(model=ko_encoder, train_dataloaders=dataloader, val_dataloaders=dataloader)




GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/fritzprix/miniconda3/envs/directml/lib/python3.10/site-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
Missing logger folder: logs/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/fritzprix/miniconda3/envs/directml/lib/python3.10/site-packages/pytorch_lightning/utilities/model_summary/model_summary.py:411: UserWarning: A layer with UninitializedParameter was found. Thus, the total number of parameters detected may be inaccurate.
  warning_cache.warn(

  | Name      | Type             | Params
-----------------------------------------------
0 | embedding | Embedding        | 52.4 M
1 | enc       | GRU              | 31.5 M
2 | fc        | LazyLinear       | 0     
3 | loss

Sanity Checking: 0it [00:00, ?it/s]

/home/fritzprix/miniconda3/envs/directml/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


IndexError: too many indices for tensor of dimension 2